# Chunk External Properties Feature Example

This notebook demonstrates how to use external properties to add business-specific attributes to chunk nodes in the graph database.

## Overview

The external properties feature allows you to:
- Add any number of business properties from source metadata to chunks
- Query and filter chunks by multiple dimensions
- Support complex multi-attribute filtering
- Enable advanced analytics and reporting

## Setup

In [ ]:
from graphrag_toolkit.lexical_graph import GraphRAGConfig, LexicalGraphIndex
from llama_index.core.schema import Document
from llama_index.core.node_parser import SentenceSplitter

## Example 1: Single Property (Simple Use Case)

### Step 1: Configure Single Property

In [ ]:
# Configure a single external property
GraphRAGConfig.chunk_external_properties = {
    'article_code': 'article_id'  # chunk.article_code from metadata['article_id']
}

print(f"External properties configured: {GraphRAGConfig.chunk_external_properties}")

### Step 2: Create Documents with Metadata

In [ ]:
# Create sample documents with article_id in metadata
documents = [
    Document(
        text="""Artificial Intelligence is transforming healthcare. 
        Machine learning algorithms can now detect diseases earlier than ever before. 
        This technology is saving lives and reducing healthcare costs.""",
        metadata={
            'article_id': 'TECH-2024-001',
            'title': 'AI in Healthcare',
            'author': 'Dr. Jane Smith'
        }
    ),
    Document(
        text="""Climate change is affecting global weather patterns. 
        Scientists are observing unprecedented changes in temperature and precipitation.""",
        metadata={
            'article_id': 'ENV-2024-042',
            'title': 'Climate Change Impact',
            'author': 'Prof. John Doe'
        }
    )
]

print(f"Created {len(documents)} documents")
for doc in documents:
    print(f"  - {doc.metadata['article_id']}: {doc.metadata['title']}")

### Step 3: Query by Single Property

In [ ]:
# Query chunks by article_code
query = """
MATCH (chunk:__Chunk__ {article_code: 'TECH-2024-001'})
RETURN chunk.chunkId, chunk.article_code, chunk.value
"""

print("Query by article_code:")
print(query)

## Example 2: Multiple Properties (Advanced Use Case)

### Step 1: Configure Multiple Properties

In [ ]:
# Configure multiple external properties
GraphRAGConfig.chunk_external_properties = {
    'article_code': 'article_id',
    'document_type': 'doc_type',
    'department': 'dept_code',
    'priority': 'priority_level'
}

print("External properties configured:")
for prop, key in GraphRAGConfig.chunk_external_properties.items():
    print(f"  chunk.{prop} ← metadata['{key}']")

### Step 2: Create Documents with Multiple Metadata Fields

In [ ]:
# Create documents with rich metadata
multi_prop_docs = [
    Document(
        text="Research paper on neural networks and deep learning...",
        metadata={
            'article_id': 'RES-2024-001',
            'doc_type': 'research',
            'dept_code': 'AI-LAB',
            'priority_level': 'high',
            'author': 'Dr. Smith'
        }
    ),
    Document(
        text="Technical report on system architecture...",
        metadata={
            'article_id': 'RPT-2024-042',
            'doc_type': 'report',
            'dept_code': 'SYS-ENG',
            'priority_level': 'medium',
            'author': 'John Doe'
        }
    ),
    Document(
        text="Policy document on data governance...",
        metadata={
            'article_id': 'POL-2024-015',
            'doc_type': 'policy',
            'dept_code': 'COMPLIANCE',
            'priority_level': 'high',
            'author': 'Jane Wilson'
        }
    )
]

print(f"Created {len(multi_prop_docs)} documents with multiple properties:")
for doc in multi_prop_docs:
    print(f"  - {doc.metadata['article_id']}: type={doc.metadata['doc_type']}, dept={doc.metadata['dept_code']}, priority={doc.metadata['priority_level']}")

### Step 3: Multi-Dimensional Queries

In [ ]:
# Query 1: Find high-priority research documents
query_1 = """
MATCH (chunk:__Chunk__)
WHERE chunk.document_type = 'research'
  AND chunk.priority = 'high'
RETURN chunk.article_code, chunk.department
"""

# Query 2: Find all chunks from AI-LAB department
query_2 = """
MATCH (chunk:__Chunk__ {department: 'AI-LAB'})
RETURN chunk.article_code, chunk.document_type, chunk.value
"""

# Query 3: Analytics - Count by document type and priority
query_3 = """
MATCH (chunk:__Chunk__)
WHERE chunk.document_type IS NOT NULL
RETURN 
    chunk.document_type,
    chunk.priority,
    count(chunk) as chunk_count
ORDER BY chunk_count DESC
"""

print("Multi-dimensional queries:")
print("\n1. High-priority research:")
print(query_1)
print("\n2. AI-LAB department:")
print(query_2)
print("\n3. Analytics by type and priority:")
print(query_3)

## Example 3: Research Paper Management

Use case: Managing research papers with DOI, journal, and field

In [ ]:
# Configure for research papers
GraphRAGConfig.chunk_external_properties = {
    'doi': 'doi',
    'journal': 'journal_name',
    'year': 'publication_year',
    'field': 'research_field'
}

research_papers = [
    Document(
        text="Abstract: This paper presents advances in neural networks...",
        metadata={
            'doi': '10.1234/nn.2024.001',
            'journal_name': 'Nature AI',
            'publication_year': '2024',
            'research_field': 'Machine Learning'
        }
    ),
    Document(
        text="Abstract: Quantum computing applications in cryptography...",
        metadata={
            'doi': '10.1234/qc.2024.042',
            'journal_name': 'Science',
            'publication_year': '2024',
            'research_field': 'Quantum Computing'
        }
    )
]

print("Research papers configured with properties:")
for paper in research_papers:
    print(f"  - DOI: {paper.metadata['doi']}")
    print(f"    Journal: {paper.metadata['journal_name']}, Field: {paper.metadata['research_field']}")

In [ ]:
# Query research papers
research_query = """
MATCH (chunk:__Chunk__)
WHERE chunk.journal = 'Nature AI'
  AND chunk.year = '2024'
  AND chunk.field = 'Machine Learning'
RETURN chunk.doi, chunk.value
"""

print("Query for ML papers in Nature AI (2024):")
print(research_query)

## Example 4: Organizational Hierarchy

Use case: Corporate documents with department, division, and project tracking

In [ ]:
# Configure for organizational hierarchy
GraphRAGConfig.chunk_external_properties = {
    'department': 'dept',
    'division': 'div',
    'team': 'team',
    'project': 'project_code'
}

corporate_docs = [
    Document(
        text="Project specification for the new AI platform...",
        metadata={
            'dept': 'Engineering',
            'div': 'R&D',
            'team': 'AI-Platform',
            'project_code': 'PROJ-2024-AI-001'
        }
    ),
    Document(
        text="Security audit report for cloud infrastructure...",
        metadata={
            'dept': 'Engineering',
            'div': 'Infrastructure',
            'team': 'Security',
            'project_code': 'PROJ-2024-SEC-042'
        }
    )
]

print("Corporate documents with organizational hierarchy:")
for doc in corporate_docs:
    print(f"  - {doc.metadata['dept']} > {doc.metadata['div']} > {doc.metadata['team']}")
    print(f"    Project: {doc.metadata['project_code']}")

In [ ]:
# Query by organizational hierarchy
org_query = """
MATCH (chunk:__Chunk__)
WHERE chunk.department = 'Engineering'
  AND chunk.division = 'R&D'
RETURN chunk.project, chunk.team, count(chunk) as chunks_per_project
ORDER BY chunks_per_project DESC
"""

print("Query Engineering R&D projects:")
print(org_query)

## Example 5: Handling Partial Metadata

The feature gracefully handles cases where some metadata keys are missing

In [ ]:
# Configure multiple properties
GraphRAGConfig.chunk_external_properties = {
    'article_code': 'article_id',
    'document_type': 'doc_type',
    'department': 'dept_code'
}

# Create documents with partial metadata
partial_docs = [
    Document(
        text="Document with all properties...",
        metadata={
            'article_id': 'DOC-001',
            'doc_type': 'report',
            'dept_code': 'ENG'
        }
    ),
    Document(
        text="Document with partial properties...",
        metadata={
            'article_id': 'DOC-002',
            'doc_type': 'memo'
            # Missing 'dept_code'
        }
    ),
    Document(
        text="Document with minimal properties...",
        metadata={
            'article_id': 'DOC-003'
            # Missing 'doc_type' and 'dept_code'
        }
    )
]

print("Documents with varying metadata:")
for doc in partial_docs:
    props = [k for k in ['article_id', 'doc_type', 'dept_code'] if k in doc.metadata]
    print(f"  - {doc.metadata.get('article_id', 'N/A')}: has {props}")

print("\nResult: Only available properties will be added to chunks.")
print("No errors will be raised for missing keys.")

## Example 6: Disabling the Feature

In [ ]:
# Disable external properties feature
GraphRAGConfig.chunk_external_properties = None

print(f"External properties feature disabled: {GraphRAGConfig.chunk_external_properties}")
print("\nChunks created after this will NOT include external properties.")

## Example 7: Advanced Analytics

Use external properties for complex analytics and reporting

In [ ]:
# Analytics query 1: Distribution by multiple dimensions
analytics_1 = """
MATCH (chunk:__Chunk__)
WHERE chunk.document_type IS NOT NULL
  AND chunk.department IS NOT NULL
RETURN 
    chunk.document_type,
    chunk.department,
    count(chunk) as chunk_count,
    count(DISTINCT chunk.article_code) as article_count
ORDER BY chunk_count DESC
"""

# Analytics query 2: Find related content
analytics_2 = """
MATCH (chunk1:__Chunk__ {article_code: 'RES-2024-001'})
MATCH (chunk2:__Chunk__)
WHERE chunk2.department = chunk1.department
  AND chunk2.article_code <> chunk1.article_code
RETURN DISTINCT chunk2.article_code, chunk2.document_type
"""

# Analytics query 3: Aggregation with filtering
analytics_3 = """
MATCH (chunk:__Chunk__)
WHERE chunk.priority = 'high'
RETURN 
    chunk.department,
    count(DISTINCT chunk.article_code) as high_priority_docs,
    count(chunk) as total_chunks
ORDER BY high_priority_docs DESC
"""

print("Advanced Analytics Queries:")
print("\n1. Distribution by type and department:")
print(analytics_1)
print("\n2. Find related content:")
print(analytics_2)
print("\n3. High-priority documents by department:")
print(analytics_3)

## Summary

The external properties feature provides:

### Key Benefits

1. **Flexible**: Add any number of business properties
2. **Multi-dimensional**: Query by multiple attributes simultaneously
3. **Graceful**: Handles missing metadata without errors
4. **Powerful**: Enables complex analytics and filtering
5. **Domain-specific**: Tailor to your business needs

### Common Use Cases

- **Document Management**: article_code, document_type, status
- **Research Papers**: doi, journal, year, field
- **Corporate Docs**: department, division, team, project
- **Compliance**: classification, retention_policy, compliance_tag
- **Content Management**: content_id, status, version, author

### Best Practices

- Start with 2-3 essential properties
- Use clear, descriptive property names
- Validate metadata before building graph
- Document your property mapping
- Consider property cardinality for queries

### Next Steps

- Define properties for your domain
- Add metadata to your documents
- Build the graph with configured properties
- Create queries leveraging multiple properties
- Combine with vector search for hybrid retrieval